In [2]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

# Define a convolution neural network
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*10*10, 10)

    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))     
        output = output.view(-1, 24*10*10)
        output = self.fc1(output)

        # self.fc1(self.bn5(self.conv5(self.bn4(self.conv4(self.pool(self.bn2(self.conv2(self.bn1(self.conv1(x))))))))))
        return output

# Instantiate a neural network model 
model = Network()

from torch.optim import Adam 
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

ModuleNotFoundError: No module named 'torchvision'

In [3]:
from torch.autograd import Variable

# Function to save the model
def saveModel():
    path = "./myFirstModel.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i % 1000 == 999:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [51]:
# parth='/Desktop/Falling/none.png'
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import sys
import os
import shutil
from distutils.dir_util import copy_tree



"""
    os.lisdr(parh) # full element in the dictionalry
    or ->  [f for f in os.listdir(path) if f.endswith('.png')]

        if we want to join something from the previus path:
    # os.rename(os.path.join(path, file), os.path.join(path, new_filename))
    # shutil.move(os.path.join(path,file),os.path.join(path,new_filename ))

    # for copy the folder or file
    copy_tree(path,new_path )
    """

def Image_opt():
    copy_path="C:/Users/Erfan/Desktop/Symetric triangle/copy"
    path="C:/Users/Erfan/Desktop/Symetric triangle/"

    # Copy the original file
    copy_tree(path,copy_path)

    for index,file  in enumerate(os.listdir(path)):  

        # if file.endswith('.png'):
        #     new_filename=f'Symetric_triangle_{index}.png'
        #     os.rename(os.path.join(path, file), os.path.join(path, new_filename))

Image_opt()
    


# for index,file  in enumerate(os.listdir(path)):  
    # new_filename=f'Symetric_Triangle_{index}.png'
    # os.rename(os.path.join(path, file), os.path.join(path, new_filename))
    # shutil.move(os.path.join(path,file),os.path.join(path,new_filename ))s

DistutilsFileError: could not create 'C:/Users/Erfan/Desktop/Symetric triangle/copy\copy\Symetric_triangle_1.png': No such file or directory

In [20]:

old_path="C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy"
new="C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/"
copy_tree(old_path,new)



['C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/650_trf006eng.png',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/Bullish-Continuation-2022.jpg',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/Double-Bottom-chart-pattern-failure-to-Expanding-Triangle.png',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/download (1).jpeg',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/download (1).png',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/download (10).png',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/download (11).png',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/download (2).jpeg',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/download (2).png',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/copy/download (3).jpeg',
 'C:/Users/Erfan/Desktop/symmetrical expanding triangle - Copy/cop

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
